## Setup

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# For emptying trash after run
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')
#drive_service.files().emptyTrash().execute()

In [3]:
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/"
#folder_name = "/content/drive/MyDrive/persuasion_technique_detection/"

## Imports

In [4]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [5]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil
import os

In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from datasets import concatenate_datasets
from transformers import Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding

In [7]:
AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti


In [8]:
model_name="Random_Forest"
summary_dir_path = folder_name + "subtask1/transformer/summaries/ensembling/summary_ensembling_train_try_2_" + model_name.replace("/","_") + "/"
train_set = 'train'

## Base Models

#### Data

In [9]:
# Initialize a counter for empty label samples
empty_label_counter = [0]

def filter_every_second_empty_label(sample):
    if len(sample['labels']) == 0:
        empty_label_counter[0] += 1

        return empty_label_counter[0] % 2 != 0
    return True

In [10]:
val_path=folder_name+"data/subtask1/validation.json"
train_path = folder_name+"data/subtask1/train.json"
test_path=folder_name+"data/subtask1/dev_subtask1_en.json"
train_added_ptc = folder_name+"data/ptc/ptc_added_train.json"
train_val_path = folder_name+"data/subtask1/train_val.json"

#dataset_memes_2024_files = {"train": train_added_ptc,"validation": val_path}
dataset_memes_2024_files = {"train": train_path,"validation": val_path}
train_files={"train":train_path}
val_files={"val":val_path}
test_files={"test":test_path}
train_val_files={"train_val":train_val_path}

dataset_memes_2024 = load_dataset("json",data_files=dataset_memes_2024_files)

empty_labels = 1
if empty_labels == 0:
  dataset_memes_2024 = dataset_memes_2024.filter(lambda x : len(x['labels']) != 0)
if empty_labels == 0.5:
  dataset_memes_2024 = dataset_memes_2024.filter(filter_every_second_empty_label)

dataset_test=load_dataset("json",data_files=test_files)
dataset_val=load_dataset("json",data_files=val_files)
dataset_train=load_dataset("json", data_files=train_files)
dataset_train_val=load_dataset("json", data_files=train_val_files)

### Multi-label

### Login to WandB

In [11]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_multilabel_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

wandb: Currently logged in as: mahmudfami (tumnlp). Use `wandb login --relogin` to force relogin


In [12]:
#checkpoint_multilabel = "vinai/bertweet-base"
checkpoint_multilabel = "microsoft/deberta-v3-base"

In [13]:
techniques = [['Black-and-white Fallacy/Dictatorship', 'Loaded Language',
       'Glittering generalities (Virtue)', 'Thought-terminating cliché',
       'Whataboutism', 'Slogans', 'Causal Oversimplification', 'Smears',
       'Name calling/Labeling', 'Appeal to authority',
       'Exaggeration/Minimisation', 'Repetition', 'Flag-waving',
       'Appeal to fear/prejudice', 'Reductio ad hitlerum', 'Doubt',
       "Misrepresentation of Someone's Position (Straw Man)",
       'Obfuscation, Intentional vagueness, Confusion', 'Bandwagon',
       'Presenting Irrelevant Data (Red Herring)']]
num_labels = len(techniques[0])
num_labels

20

In [14]:
mlb = MultiLabelBinarizer()
mlb.fit(techniques)

MultiLabelBinarizer()

#### Inference of val set

In [15]:
tokenizer_multilabel = AutoTokenizer.from_pretrained(checkpoint_multilabel)
data_collator_multilabel = DataCollatorWithPadding(tokenizer=tokenizer_multilabel)
def tokenize_multilabel(examples):
    encoding = tokenizer_multilabel(examples["text"], truncation=True,padding=True)#,max_length=512)
    return encoding

/home/nlp-lab-ws23/nlp_praktikum/nlp_praktikum_env/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [16]:
def get_preds_multilabel(binarizer,predicted_logits,threshold=0.5):

  predicted_logits_tensor = torch.from_numpy(predicted_logits)
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(predicted_logits_tensor)
  preds=(probs>threshold).int()

  mask = (preds == 1)

  # Use the mask to select the corresponding probabilities
  selected_probs_list = []

  # Iterate over rows and select probabilities using the mask
  for i in range(preds.size(0)):
      selected_probs_row = probs[i][mask[i]].tolist()
      selected_probs_list.append(selected_probs_row)

  names=binarizer.inverse_transform(preds)
  res=[(list(x), y) for x, y in zip(names, selected_probs_list)]
  return res

In [17]:
project_name_multilabel="subtask1_multilabel_transformer_encoder_classification"

def return_trainer_multilabel(model_name,unique_labels):
  api = wandb.Api()
  artifact=api.artifact(model_name)
  model_dir=artifact.download()
  model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]), problem_type="multi_label_classification", ignore_mismatched_sizes=True)

  trainer = Trainer(model=model)
  if torch.cuda.is_available():
    trainer.model = model.cuda()
  return trainer

In [18]:
if train_set == 'train': 
    prediction_set=dataset_memes_2024['train']
elif train_set == 'train_val': 
    prediction_set=dataset_train_val['train_val']
else:
    prediction_set=dataset_val['val']
prediction_set=prediction_set.remove_columns(["labels"])

In [19]:
best_model_multilabel = "model-microsoft_deberta-v3-base-multilabel-memes_0.4threshold_5e-05learningRate:v5"
best_threshold_multilabel = 0.0

In [20]:
trainer=return_trainer_multilabel(best_model_multilabel, techniques)
threshold=best_threshold_multilabel
prediction_set_tokenized=prediction_set.map(tokenize_multilabel, batched=True)

preds=get_preds_multilabel(mlb,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds_multilabel_train=dict(zip(prediction_set_tokenized["id"],preds))

wandb: Downloading large artifact model-microsoft_deberta-v3-base-multilabel-memes_0.4threshold_5e-05learningRate:v5, 714.21MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.0


In [21]:
final_ds_multilabel_train

{'65635': (['Appeal to authority',
   'Appeal to fear/prejudice',
   'Bandwagon',
   'Black-and-white Fallacy/Dictatorship',
   'Causal Oversimplification',
   'Doubt',
   'Exaggeration/Minimisation',
   'Flag-waving',
   'Glittering generalities (Virtue)',
   'Loaded Language',
   "Misrepresentation of Someone's Position (Straw Man)",
   'Name calling/Labeling',
   'Obfuscation, Intentional vagueness, Confusion',
   'Presenting Irrelevant Data (Red Herring)',
   'Reductio ad hitlerum',
   'Repetition',
   'Slogans',
   'Smears',
   'Thought-terminating cliché',
   'Whataboutism'],
  [1.790695205272641e-05,
   0.0009059724980033934,
   7.348461076617241e-05,
   0.9952859282493591,
   0.000507467717397958,
   0.0002792977320495993,
   6.882331490487559e-06,
   4.378381709102541e-05,
   8.581117435824126e-05,
   6.022732122801244e-05,
   4.1440463974140584e-05,
   1.5259950032486813e-06,
   6.456406117649749e-05,
   2.7421813229011605e-06,
   8.333662844961509e-05,
   7.980126611073501e-

### Hierarchical

Login to WandB

In [22]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

In [23]:
checkpoint_hierarchical="vinai/bertweet-large"
#checkpoint_hierarchical = "microsoft/deberta-v3-large"

In [24]:
lvl1_parents={"Appeal to authority":["Ethos","Logos"],
              "Appeal to fear/prejudice":["Pathos","Logos"],
              "Bandwagon":["Ethos","Logos"],
              "Black-and-white Fallacy/Dictatorship":["Logos"],
              "Causal Oversimplification":["Logos"],
              "Doubt":["Ethos"],
              "Exaggeration/Minimisation":["Pathos"],
              "Flag-waving":["Pathos","Logos"],
              "Glittering generalities (Virtue)":["Ethos"],
              "Loaded Language":["Pathos"],
              "Misrepresentation of Someone's Position (Straw Man)":["Logos"],
              "Name calling/Labeling":["Ethos"],
              "Obfuscation, Intentional vagueness, Confusion":["Logos"],
              "Presenting Irrelevant Data (Red Herring)":["Logos"],
              "Reductio ad hitlerum":["Ethos"],
              "Repetition":["Logos"],
              "Slogans":["Logos"],
              "Smears":["Ethos"],
              "Thought-terminating cliché":["Logos"],
              "Whataboutism":["Ethos","Logos"]}

lvl2_parents={"Appeal to authority":["Justification"],
              "Appeal to fear/prejudice":["Justification"],
              "Bandwagon":["Justification"],
              "Black-and-white Fallacy/Dictatorship":["Reasoning"],
              "Causal Oversimplification":["Reasoning"],
              "Doubt":["Ad Hominem"],
              "Exaggeration/Minimisation":[],
              "Flag-waving":["Justification"],
              "Glittering generalities (Virtue)":[],
              "Loaded Language":[],
              "Misrepresentation of Someone's Position (Straw Man)":["Reasoning"],
              "Name calling/Labeling":["Ad Hominem"],
              "Obfuscation, Intentional vagueness, Confusion":[],
              "Presenting Irrelevant Data (Red Herring)":["Reasoning"],
              "Reductio ad hitlerum":["Ad Hominem"],
              "Repetition":[],
              "Slogans":["Justification"],
              "Smears":["Ad Hominem"],
              "Thought-terminating cliché":["Reasoning"],
              "Whataboutism":["Ad Hominem","Reasoning"]}

lvl3_parents={"Appeal to authority":[],
              "Appeal to fear/prejudice":[],
              "Bandwagon":[],
              "Black-and-white Fallacy/Dictatorship":["Simplification"],
              "Causal Oversimplification":["Simplification"],
              "Doubt":[],
              "Exaggeration/Minimisation":[],
              "Flag-waving":["Justification"],
              "Glittering generalities (Virtue)":[],
              "Loaded Language":[],
              "Misrepresentation of Someone's Position (Straw Man)":["Distraction"],
              "Name calling/Labeling":[],
              "Obfuscation, Intentional vagueness, Confusion":[],
              "Presenting Irrelevant Data (Red Herring)":["Distraction"],
              "Reductio ad hitlerum":[],
              "Repetition":[],
              "Slogans":[],
              "Smears":[],
              "Thought-terminating cliché":["Simplification"],
              "Whataboutism":["Distraction"]}


def create_parent_labels(data):
  labels=data["labels"]
  persuasion=[]
  ethos=[]
  pathos=[]
  logos=[]
  ad_hominem=[]
  justification=[]
  reasoning=[]
  distraction=[]
  simplification=[]
  persuasion_or_not=[]

  if len(labels)>0:
    persuasion_or_not.append("Persuasion")
    for el in labels:

      if "Ethos" in lvl1_parents[el]:
        persuasion.append("Ethos")
        if len(lvl2_parents[el])>0 and "Ad Hominem" in lvl2_parents[el]:
          ethos.append("Ad Hominem")
          ad_hominem.append(el)
        else:
          ethos.append(el)
      if "Pathos" in lvl1_parents[el]:
        persuasion.append("Pathos")
        pathos.append(el)
      if "Logos" in lvl1_parents[el]:
        persuasion.append("Logos")
        if len(lvl2_parents[el])>0 and "Justification" in lvl2_parents[el]:
          logos.append("Justification")
          justification.append(el)
        elif len(lvl2_parents[el])>0 and "Reasoning" in lvl2_parents[el]:
          logos.append("Reasoning")
          reasoning.append(lvl3_parents[el][0])
          if lvl3_parents[el][0]=="Simplification":
            simplification.append(el)
          else:

            distraction.append(el)
        else:
          logos.append(el)
  else:
    persuasion_or_not.append("Not Persuasion")


  data["persuasion"]=list(set(persuasion))
  data["ethos"]=list(set(ethos))
  data["pathos"]=list(set(pathos))
  data["logos"]=list(set(logos))
  data["ad_hominem"]=list(set(ad_hominem))
  data["justification"]=list(set(justification))
  data["reasoning"]=list(set(reasoning))
  data["distraction"]=list(set(distraction))
  data["simplification"]=list(set(simplification))
  data["persuasion_or_not"]=persuasion_or_not
  return data

dataset_memes_2024=dataset_memes_2024.map(create_parent_labels)

In [25]:
persuasion_unique=[["Ethos","Pathos","Logos"]]
ethos_unique=[["Ad Hominem","Bandwagon","Appeal to authority","Glittering generalities (Virtue)"]]
pathos_unique=[["Exaggeration/Minimisation","Loaded Language","Flag-waving","Appeal to fear/prejudice"]]
logos_unique=[["Justification","Reasoning","Repetition","Obfuscation, Intentional vagueness, Confusion"]]
ad_hominem_unique=[["Name calling/Labeling","Doubt","Smears","Reductio ad hitlerum","Whataboutism"]]
justification_unique=[["Bandwagon","Appeal to authority","Flag-waving","Appeal to fear/prejudice","Slogans"]]
reasoning_unique=[["Distraction","Simplification"]]
distraction_unique=[["Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)","Whataboutism"]]
simplification_unique=[["Causal Oversimplification","Black-and-white Fallacy/Dictatorship","Thought-terminating cliché"]]
persuasion_or_not_unique=[["Not Persuasion","Persuasion"]]

In [26]:
mlb_persuasion = MultiLabelBinarizer()
mlb_persuasion.fit(persuasion_unique)

mlb_ethos = MultiLabelBinarizer()
mlb_ethos.fit(ethos_unique)

mlb_pathos = MultiLabelBinarizer()
mlb_pathos.fit(pathos_unique)

mlb_logos = MultiLabelBinarizer()
mlb_logos.fit(logos_unique)

mlb_ad_hominem = MultiLabelBinarizer()
mlb_ad_hominem.fit(ad_hominem_unique)

mlb_justification = MultiLabelBinarizer()
mlb_justification.fit(justification_unique)

mlb_reasoning=MultiLabelBinarizer()
mlb_reasoning.fit(reasoning_unique)

mlb_distraction=MultiLabelBinarizer()
mlb_distraction.fit(distraction_unique)

mlb_simplification=MultiLabelBinarizer()
mlb_simplification.fit(simplification_unique)

lb_persuasion_or_not=LabelEncoder()
lb_persuasion_or_not.fit(persuasion_or_not_unique[0])


LabelEncoder()

In [27]:
tokenizer_hierarchical = AutoTokenizer.from_pretrained(checkpoint_hierarchical)
data_collator_hierarchical = DataCollatorWithPadding(tokenizer=tokenizer_hierarchical)
def tokenize_hierarchical(examples):
    encoding = tokenizer_hierarchical(examples["text"], truncation=True ,padding=True, max_length=512)
    return encoding

In [28]:
def get_preds_hierarchical(binarizer,predicted_logits,threshold=0.5):

  predicted_logits_tensor = torch.from_numpy(predicted_logits)

  if isinstance(binarizer,LabelEncoder):
    softmax=torch.nn.Softmax(dim=1)
    full_probs=softmax(predicted_logits_tensor)

    probs,predictions=torch.max(full_probs,dim=1)
    predictions=[[p] for p in binarizer.inverse_transform(predictions)]
    probs = [[value.item()] for value in probs]
    res=[(x, y) for x, y in zip(predictions, probs)]
    return res
  else:
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(predicted_logits_tensor)
    preds=(probs>threshold).int()

    mask = (preds == 1)

    # Use the mask to select the corresponding probabilities
    selected_probs_list = []

    # Iterate over rows and select probabilities using the mask
    for i in range(preds.size(0)):
        selected_probs_row = probs[i][mask[i]].tolist()
        selected_probs_list.append(selected_probs_row)

    names=binarizer.inverse_transform(preds)
    res=[(list(x), y) for x, y in zip(names, selected_probs_list)]
    return res

In [29]:
project_name_hierarchical="subtask1_transformer_encoder_classification"

def return_trainer_hierarchical(model_name,unique_labels):
  api = wandb.Api()
  artifact=api.artifact(model_name)
  model_dir=artifact.download()
  if "persuasion_or_not" in model_name:
    lbls=unique_labels[0]
    id2label={0:lbls[0],1:lbls[1]}
    label2id={lbls[0]:0,lbls[1]:1}
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]),id2label=id2label, label2id=label2id)
  else:
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]), problem_type="multi_label_classification", ignore_mismatched_sizes=True)

  trainer = Trainer(model=model)
  if torch.cuda.is_available():
    trainer.model = model.cuda()
  return trainer

In [30]:
if train_set == 'train':
    prediction_set=dataset_memes_2024["train"]
elif train_set == 'train_val': 
    prediction_set=dataset_train_val['train_val']
else: 
    prediction_set=dataset_memes_2024["validation"]
prediction_set=prediction_set.remove_columns(["labels"])

In [31]:
def get_new_labels(model_address,node,unique,threshold,binarizer):
  trainer=return_trainer_hierarchical(model_address,unique)
  preds=get_preds_hierarchical(binarizer,trainer.predict(curr_set).predictions,threshold)

  for i in range(len(preds)):

    if len(preds[i][0])>0:

      t=prev_labels[i][0].index(node)
      _=prev_labels[i][1].pop(t)
      prev_labels[i][0].remove(node)
      parent_labels=prev_labels[i][0]
      parent_probs=prev_labels[i][1]

      new_labels.append((parent_labels+preds[i][0],parent_probs+preds[i][1]))
    else:
      new_labels.append(prev_labels[i])

In [32]:
model_nodes = {
    "persuasion_or_not": "model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v3",
    "persuasion": "model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v2",
    "ethos": "model-vinai_bertweet-large-ethos-memes_0.4threshold_5e-06learningRate:v2",
    "pathos": "model-vinai_bertweet-large-pathos-memes_0.3threshold_5e-06learningRate:v2",
    "logos": "model-vinai_bertweet-large-logos-memes_0.5threshold_5e-06learningRate:v2",
    "ad_hominem": "model-vinai_bertweet-large-ad_hominem-memes_0.3threshold_5e-06learningRate:v2",
    "justification": "model-vinai_bertweet-large-justification-memes_0.4threshold_5e-06learningRate:v2",
    "reasoning": "model-vinai_bertweet-large-reasoning-memes_0.3threshold_5e-06learningRate:v2",
    "simplification": "model-vinai_bertweet-large-simplification-memes_0.4threshold_5e-06learningRate:v2",
    "distraction": "model-vinai_bertweet-large-distraction-memes_0.5threshold_0.0005learningRate:v2"
  }

threshold_nodes = {
    "persuasion_or_not": 0,
    "persuasion": 0,
    "ethos": 0,
    "pathos": 0,
    "logos": 0,
    "ad_hominem": 0,
    "justification": 0,
    "reasoning": 0,
    "simplification": 0,
    "distraction": 0
}

Leaf nodes that have to entry edges have two probabilities

In [33]:
trainer=return_trainer_hierarchical(model_nodes["persuasion"],persuasion_unique)
threshold=threshold_nodes['persuasion']
prediction_set_tokenized=prediction_set.map(tokenize_hierarchical ,batched=True)

preds=get_preds_hierarchical(mlb_persuasion,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds_hierarchical_train=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds_hierarchical_train.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds_hierarchical_train[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes[node_names[i]],node,unique_node_labels[i],threshold_nodes[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds_hierarchical_train[ids[k]]=new_labels[k]


wandb: Downloading large artifact model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v2, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.7


Node: Ethos, curr_set: Dataset({
    features: ['text', 'link', 'id', 'persuasion', 'ethos', 'pathos', 'logos', 'ad_hominem', 'justification', 'reasoning', 'distraction', 'simplification', 'persuasion_or_not', 'input_ids', 'attention_mask'],
    num_rows: 5299
})


wandb: Downloading large artifact model-vinai_bertweet-large-ethos-memes_0.4threshold_5e-06learningRate:v2, 1358.87MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.8


ValueError: expected sequence of length 360 at dim 1 (got 348)

In [ ]:
# Removing duplicate labels and sorting labels alphabetically
for id, (labels, scores) in final_ds_hierarchical_train.items():
    label_score_dict = {}
    for label, score in zip(labels, scores):
        if label not in label_score_dict or label_score_dict[label] < score:
            label_score_dict[label] = score

    sorted_items = sorted(label_score_dict.items())
    unique_labels = [item[0] for item in sorted_items]
    max_scores = [item[1] for item in sorted_items]

    final_ds_hierarchical_train[id] = (unique_labels, max_scores)


In [ ]:
final_ds_hierarchical_train

In [ ]:
## Meta-Model

### Training

Concatenate predictions to one feature set

In [ ]:
keys = final_ds_multilabel_train.keys()
probabilities_multilabel_train = np.array([final_ds_multilabel_train[key][1] for key in keys])
probabilities_hierarchical_train = np.array([final_ds_hierarchical_train[key][1] for key in keys])
combined_features_train= np.concatenate((probabilities_multilabel_train, probabilities_hierarchical_train), axis=1)
combined_features_train

Prepare Gold Labels

In [ ]:
if train_set == 'train':
    validation_set_df = pd.read_json(train_path)
elif train_set == 'train_val': 
    validation_set_df= pd.read_json(train_val_path)
else:
    validation_set_df = pd.read_json(val_path)
validation_set_df

In [ ]:
gold_labels_bin = mlb.transform(validation_set_df['labels'])
gold_labels_bin

Hyperparam Search

In [ ]:
#summary_dir_path = folder_name + "subtask1/transformer/summaries/ensembling/summary_ensembling_train_val_1_" + model_name.replace("/","_") + "/"

In [ ]:
def write_json(path,data,test=False):
  if not isinstance(data, dict):
    data = data.to_dict("records")

  with open(path, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

def hierarchical_scores(gold_label_path, pred_label_path):
    scorer = folder_name + "subtask1/subtask_1_2a.py"
    command = f'python3 {scorer} --gold_file_path {gold_label_path} --pred_file_path {pred_label_path}'

    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True)
    output = result.stdout.strip()

    parts = output.split('\t')
    f1_h = parts[0].split('=')[1]
    prec_h = parts[1].split('=')[1]
    rec_h = parts[2].split('=')[1]

    f1_h = float(f1_h)
    prec_h = float(prec_h)
    rec_h = float(rec_h)

    hierarchical_metrics = {"f1_hierarchical": f1_h, "precision_hierarchical": prec_h, "recall_hierarchical": rec_h}
    return hierarchical_metrics

def compute_metrics(estimator, x, y):
    y_pred = estimator.predict(x)
    labels= y
    print(f"y_pred: {y_pred}")
    print(f"labels: {labels}")
    preds = mlb.inverse_transform(y_pred)
    pred_keys= [i for i in range(len(preds))]
    final_ds_pred_search=dict(zip(pred_keys,[list(pred) for pred in preds]))
    pred_df=pd.DataFrame.from_dict({"id":final_ds_pred_search.keys(),"labels":final_ds_pred_search.values()})
    final_ds_true_search=dict(zip(pred_keys,[list(label) for label in labels]))
    true_df=pd.DataFrame.from_dict({"id":final_ds_true_search.keys(),"labels":final_ds_true_search.values()})

    search_tmp_pred=summary_dir_path + "tmp/search_tmp_pred.json"
    if not os.path.exists(summary_dir_path):
        os.makedirs(summary_dir_path)

    write_json(search_tmp_pred,pred_df)

    search_tmp_true=summary_dir_path + "tmp/search_tmp_true.json"
    write_json(search_tmp_true,true_df)

    scores = hierarchical_scores(search_tmp_true,search_tmp_pred)
    print(f"Scores: {scores}")

    return scores['f1_hierarchical']

In [ ]:
def search_random_forest():
    params = {
        "n_estimators": [500],
        #"max_depth": [30, 40, 50, 75, 100],
        "min_samples_leaf":  [1e-08, 1e-07, 1e-06, 1e-05],
        "max_features":  [0.7, 0.8, 0.9, 1.0],
    }
    model = RandomForestClassifier()
    search = GridSearchCV(
        model, params, cv=2, n_jobs=-1, verbose=10, scoring="f1_weighted"
    )
    return search

search = search_random_forest()
search = search.fit(combined_features_train, gold_labels_bin)
meta_model = search.best_estimator_

In [ ]:
import pickle
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)
with open(os.path.join(summary_dir_path, 'cvresults.pkl'), "wb") as f:
    pickle.dump(search.cv_results_, f)

In [ ]:
cv_results = pd.DataFrame(search.cv_results_)[['param_max_features', 'param_min_samples_leaf', 'param_n_estimators', 'mean_test_score', 'rank_test_score']]
cv_results

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
cv_results.sort_values('rank_test_score')

### Prediction

In [ ]:
prediction_set=dataset_test['test']
prediction_set=prediction_set.remove_columns(["labels"])

Multi-label

In [ ]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_multilabel_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

In [ ]:
trainer=return_trainer_multilabel(best_model_multilabel, techniques)
threshold=best_threshold_multilabel
prediction_set_tokenized=prediction_set.map(tokenize_multilabel, batched=True)

preds=get_preds_multilabel(mlb,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds_multilabel_pred=dict(zip(prediction_set_tokenized["id"],preds))

Hierarchical

In [ ]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

In [ ]:
dataset_test = dataset_test.map(create_parent_labels)
prediction_set=dataset_test['test']
prediction_set=prediction_set.remove_columns(["labels"])

In [ ]:
trainer=return_trainer_hierarchical(model_nodes["persuasion"],persuasion_unique)
threshold=threshold_nodes['persuasion']
prediction_set_tokenized=prediction_set.map(tokenize_hierarchical ,batched=True)

preds=get_preds_hierarchical(mlb_persuasion,trainer.predict(prediction_set_tokenized).predictions,threshold)
final_ds_hierarchical_pred=dict(zip(prediction_set_tokenized["id"],preds))

model_seq=["Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds_hierarchical_pred.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds_hierarchical_pred[x] for x in ids]
  new_labels=[]
  curr_set=prediction_set_tokenized.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  get_new_labels(model_nodes[node_names[i]],node,unique_node_labels[i],threshold_nodes[node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds_hierarchical_pred[ids[k]]=new_labels[k]

In [ ]:
# Removing duplicate labels and sorting labels alphabetically
for id, (labels, scores) in final_ds_hierarchical_pred.items():
    label_score_dict = {}
    for label, score in zip(labels, scores):
        if label not in label_score_dict or label_score_dict[label] < score:
            label_score_dict[label] = score

    sorted_items = sorted(label_score_dict.items())
    unique_labels = [item[0] for item in sorted_items]
    max_scores = [item[1] for item in sorted_items]

    final_ds_hierarchical_pred[id] = (unique_labels, max_scores)

Concatenate predictions

In [ ]:
keys = final_ds_multilabel_pred.keys()
probabilities_multilabel_pred = np.array([final_ds_multilabel_pred[key][1] for key in keys])
probabilities_hierarchical_pred = np.array([final_ds_hierarchical_pred[key][1] for key in keys])
combined_features_pred= np.concatenate((probabilities_multilabel_pred, probabilities_hierarchical_pred), axis=1)
combined_features_pred

In [ ]:
# Predict on the test set
y_pred = meta_model.predict(combined_features_pred)

Get Labels

In [ ]:
preds = mlb.inverse_transform(y_pred)
final_ds_pred=dict(zip(prediction_set_tokenized["id"],[list(pred) for pred in preds]))
final_ds_pred

### Evaluate using scorer script

In [ ]:
pred_df=pd.DataFrame.from_dict({"id":final_ds_pred.keys(),"labels":final_ds_pred.values()})

dev_pred_file=summary_dir_path + "dev_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(dev_pred_file,pred_df)

scores = hierarchical_scores(test_path,dev_pred_file)
scores

Delete tmp files

In [ ]:
def delete_dir(path):
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"The directory {path} and all its contents have been deleted successfully")
        except OSError as e:
            print(f"Error: {e.strerror}")
    else:
        print(f"The directory {path} does not exist")

In [ ]:
def delete_output_dirs(parent_directory):
    for entry in os.listdir(parent_directory):
        entry_path = os.path.join(parent_directory, entry)

        if os.path.isdir(entry_path) and entry.startswith('output_'):
            delete_dir(entry_path)

In [ ]:
summary = {
    "checkpoint_multilabel" : checkpoint_multilabel,
    "checkpoint_hierarchical" : checkpoint_hierarchical,
    "training_set": train_set,
    "empty_labels": empty_labels,
    "best_model_multilabel" : best_model_multilabel,
    "best_models_hierarchical": model_nodes,
    "meta_model": str(meta_model),
    "hierarchical_scores": scores
}

summary

In [ ]:
summary_file_path = summary_dir_path + "summary.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)
write_json(summary_file_path, summary)

In [ ]:
path = folder_name + "subtask1"
delete_output_dirs(path)
path = path + "/transformer/ensembling/"
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")
path = summary_dir_path + "tmp"
